# Больше практики.

In [ ]:
from __future__ import division, print_function
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

### Попытаемся ответить на вопрос, что делать, если качество модели нас не устраивает?

Есть несколько путей решения:

a) добавить признаков

б) добавить данных

в) усложнить/упростить модель

Изначально не очевидно, какой (или какие) подходы позволят улучшить качество модели.

Посмотрим на пример. Будем работать с данными по оттоку клиентов телеком-оператора.

In [ ]:
data = pd.read_csv('telecom_churn.csv').drop('State', axis=1)
data.head()

Преобразуйте колонку International plan таким образом, чтобы вместо Yes в ней стояла 1, а вместо No - 0.

Аналогичным образом преобразуйте колонку Voice mail plan.

Затем преобразуйте значения колонки Churn (целевой столбец) в 1 (если True) и 0 (если False).

In [ ]:
#your code here

In [ ]:
data.head()

Создайте матрицу объект-признак из таблицы data и целевой вектор (data['Churn']), затем выкиньте из матрицы вектор 'Churn'.

In [ ]:
#your code here
y = ...
X = ...

X.shape, y.shape

Перед применением линейной модели **необходимо масштабировать признаки**. Создадим пайплайн, в котором сначала происходит масштабирование, а затем применяется модель. В данном случае будем использовать логистическую регрессию.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

logit_pipe = Pipeline([('scaler', StandardScaler()), \
                       ('logit', LogisticRegression(class_weight='balanced'))])

Обучите модель (logit_pipe) по кросс-валидации и выведите на экран roc-auc (используйте функцию cross_val_score).

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

#your code here

Как правило, подбор гиперпараметров алгоритма улучшает его качество. Подберём значение параметра регуляризации C в логистической регрессии по кросс-валидации, используя GridSearchCV.

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

param_grid = {'logit__C': np.logspace(-2, 0, 20)}

#your code here

In [ ]:
grid_logit.best_params_, grid_logit.best_score_

Попробуем улучшить модель путём добавления новых признаков. **Добавьте в модель все квадратичные признаки, используя функцию PolynomialFeatures**.

Тогда происходит три действия: масштабирование, добавление признаков и применение модели. Объедините их в один пайплайн.

In [ ]:
#your code here
logit_pipe2 = ...

Подберите значение параметра C по кросс-валидации

In [ ]:
%%time

param_grid = {'logit__C': np.logspace(-2, 0, 20)}

#your code here
grid_logit2 = ...
grid_logit2.fit(X, y)

In [ ]:
grid_logit2.best_params_, grid_logit2.best_score_

### Комментарии

1. Ошибка на обучающей выборке сама по себе ничего не говорит о качестве модели

2. Кросс-валидационная ошибка показывает, насколько хорошо модель подстраивается под данные (имеющийся тренд в данных), сохраняя при этом способность обобщения на новые данные

3. В данной задаче качество модели улучшилось при добавлении квадратичных признаков.

### Применим метод K ближайших соседей для решения задачи.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()

cross_val_score(knn_model, X, y, cv=3, scoring='roc_auc').mean()

Подберите число соседей n_neighbors в методе с помощью GridSearchCV. Какое число соседей получилось оптимальным? Какое качество показал алгоритм на кросс-валидации?

In [ ]:
%%time

#your code here

Добавьте квадратичных признаков в модель и создайте пайплайн из добавления признаков и применения метода KNN. Найдите оптимальное количество соседей по кросс-валидации. Какое количество соседей получилось в этой модели? Какое качество показал алгоритм на кросс-валидации?

In [ ]:
#your code here

# Задание.

Примените метод опорных векторов для решения данной задачи. 

Используйте пайплайн, состоящий из масштабирования, извлечения квадратичных признаков и применения SVM.

Вычислите качество алгоритма на кросс-валидации.

In [ ]:
from sklearn.svm import SVC

#your code here

Подберите значение параметра регуляризации C по кросс-валидации и нарисуйте валидационную кривую, отражающую качество в зависимости от C. Сделайте выводы.

In [ ]:
#your code here

Посмотрите на количество 0 и 1 в целевой переменной. Является ли выборка сбалансированной?

In [ ]:
#your code here

Добавьте в обученную выше последнюю версию логистической регрессии параметр **class_weight = 'balanced'**. Посмотрите на качество на кросс-валидации.

In [ ]:
#your code here

Аналогично - добавьте в модель SVM параметр class_weight = 'balanced' и выведите на экран качество алгоритма на кросс-валидации.

In [ ]:
#your code here

Посмотрим, улучшит ли качество применение SVM с ядром. Попробуйте использовать SVM с различными ядрами для данной задачи. 

Для каждого ядра подберите значение параметра C по кросс-валидации.

Для наилучшего из алгоритмов выведите график learning curve.

In [ ]:
#your code here

# Решающие деревья

Рассмотрим модельную задачу регрессии. Объектами будут являться точки на плоскости (т.е. каждый объект описывается 2 признаками), целевая переменная — расстояние от объекта до точки (0, 0).

Напишем вспомогательную функцию, которая будет возвращать решетку для дальнейшей красивой визуализации

In [ ]:
def get_grid(data):
    x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
    y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
    return np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))

Сгенерируем выборку

In [ ]:
data_x = np.random.normal(size=(100, 2))
data_y = (data_x[:, 0] ** 2 + data_x[:, 1] ** 2) ** 0.5
plt.figure(figsize=(8, 8))
plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=100, cmap='spring')

Обучим дерево на сгенерированных данных и предскажем ответы для каждой точки решетки

In [ ]:
from sklearn.tree import DecisionTreeRegressor

clf = DecisionTreeRegressor()
clf.fit(data_x, data_y)

xx, yy = get_grid(data_x)
print(np.c_[xx.ravel(), yy.ravel()])

predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

plt.figure(figsize=(8, 8))
plt.pcolormesh(xx, yy, predicted, cmap='spring')
plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=100, cmap='spring')

### Задание

Сейчас мы сгенерировали 100 точек из нормального распределения и обучили решающее дерево на них. Сгенерируйте 300 точек из нормального распределения, обучите на них дерево и выведите на экран результат (как на картинке выше).

Сгенерированные точки и расстояние до точек сохраните в массивы data_x300, data_y300, для обучения и предсказания используйте эти массивы.

Улучшилось ли предсказание алгоритма на решётке? (т.е. стала ли раскраска всей плоскости более правильной?)

In [ ]:
#your code here

Вернёмся к исходным данным (100 точек).

Посмотрим как будут выглядеть разделяющая поверхность в зависимости от 
- минимального количества объектов в листе
- максимальной глубины дерева

In [ ]:
plt.figure(figsize=(18, 18))
for i, max_depth in enumerate([1, 2, 4, 6]):
    for j, min_samples_leaf in enumerate([1, 5, 10, 15]):
        clf = DecisionTreeRegressor(max_depth=max_depth, min_samples_leaf=min_samples_leaf)
        clf.fit(data_x, data_y)
        xx, yy = get_grid(data_x)
        predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
        
        plt.subplot2grid((4, 4), (i, j))
        plt.pcolormesh(xx, yy, predicted, cmap='spring')
        plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=30, cmap='spring')
        plt.title('max_depth=' + str(max_depth) + ', min_samples_leaf: ' + str(min_samples_leaf))

- Как влияет увеличение максимальной глубины и/или уменьшение минимального количества объектов выборки в листе на качество на обучающей выборке? на переобучение?

## Неустойчивость решающих деревьев

Решающие деревья — это алгоритмы, неустойчивые к изменениям обучающей выборки, т.е. при малейших её изменениях итоговый классификатор может радикально измениться.
Посмотрим, как будет меняться структура дерева при обучении на разных 90%-х подвыборках.

In [ ]:
plt.figure(figsize=(20, 6))
for i in range(3):
    clf = DecisionTreeRegressor(random_state=42)

    indices = np.random.randint(data_x.shape[0], size=int(data_x.shape[0] * 0.9))
    clf.fit(data_x[indices], data_y[indices])
    xx, yy = get_grid(data_x)
    predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    plt.subplot2grid((1, 3), (0, i))
    plt.pcolormesh(xx, yy, predicted, cmap='winter')
    plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=30, cmap='winter')

## Подбор параметров

Посмотрим на качество дерева в зависимости от параметров на одном из стандартных наборов данных - Бостонском датасете.

In [ ]:
from sklearn.datasets import load_boston

data = load_boston()
print(data.DESCR)

In [ ]:
X = data.data
y = data.target

X.shape

- будем оценивать качество алгоритма по кросс-валидации

Можем зафиксировать разбиение на фолды, чтобы затем каждый раз использовать одно и то же разбиение при кросс-валидации, это полезно при сравнении алгоритмов

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
cv = KFold(X.shape[0], shuffle=True, random_state=241)

Выведите качество DecisionTreeRegressor, обученного на данных X, y по кросс-валидации. В функции cross_val_score в качестве cv поставьте cv=cv, в качестве метрики - 'neq_mean_squared_error'

In [ ]:
#your code here

### Задание

Метрика MSЕ имеет не ограничена сверху. Поэтому для оценки качества алгоритма можно также пользоваться метрикой R2 (коэффициент детерминации), так как он не превышает 1 (и чем ближе к 1, тем лучше).

Выведите на экран значение R2 алгоритма ('r2').

In [ ]:
#your code here

Для сравнения качества модели при различных наборах параметров или для сравнения моделей на одном датасете можно использовать, как и раньше, MSE.

Будем подбирать параметры решающего дерева по сетке с целью увеличить качество алгоритма. Будем подбирать значения max_features и max_depth.

In [ ]:
from sklearn.metrics import SCORERS
SCORERS.keys()

Подберите по кросс-валидации оптимальные значения max_features и max_depth. В функции GridSearchCV в качестве cv поставьте заранее фиксированное разбиение (cv=cv), метрику качества используйте scoring='neq_mean_squared_error'

In [ ]:
params={'max_features': [None, 'log2', 'sqrt'], 
        'max_depth': [2, 4, 6, 8, 10, 20, 50]},

gs = GridSearchCV(DecisionTreeRegressor(), params, cv=3, n_jobs=-1)

gs.fit(X, y)

Выведем на экран средние значения и стандартные отклонения, полученные при GridSearch.

In [ ]:
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gs.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params))

# Задание

Теперь попробуем одновременно подбирать значения max_features, max_depth и min_samples_leaf. Ищите min_samples_leaf в диапазоне от 1 до 20 с шагом 1.

In [ ]:
#your code here

In [ ]:
gs.best_score_

Как в данной задаче зависит качество алгоритма от количества параметров, которые мы оптимизируем?

# Домашнее задание
Поработайте с датасетом wine_data (в каждой строке этого датасета содержится информация о бутылках вина одного из трёх видов: в первой колонке - вид вина (1, 2 или 3), в колонках 1-13 - свойства вина). 

A. Решите задачу с помощью DecisionTreeClassifier:
- подберите гиперпараметры алгоритма по кросс-валидации
Что вы можете сказать об алгоритме? Он переобучился/недообучился/обучился как надо и вы довольны качеством?

B. Попробуйте увеличить число признаков с помощью добавления полиномиальных признаков. Также можно добавить функции от признаков вручную (те, которые вам кажутся подходящими в данной задаче).
Улучшилось ли качество алгоритма?

C. Постройте матрицу корреляций признаков. Есть ли признаки, которые практически не коррелируют с целевой переменной? Попробуйте их удалить. Есть ли пары сильно коррелирующих между собой признаков? Удалите по одному признаку из каждой пары.
Помогло ли это улучшить качество в данной задаче?

Пункты B и C можно скомбинировать для достижения наилучшего качества.

Далее попробуйте решить исходную задачу линейными методами. Не забудьте масштабировать данные перед применением этих методов:

D. Решите исходную задачу с помощью логистической регрессии.

E. Решите исходную задачу с помощью SVMClassifier с линейным и нелинейными ядрами.

F. Решите исходную задачу с помощью наивного байесовского классификатора (https://scikit-learn.org/stable/modules/naive_bayes.html).

G. Сделайте мини-отчет о проведенной работе. Для этого ответьте на вопросы:
a) какой классификатор дал наилучший результат? чему равны значения метрик MSE и R2?

b) какие новые признаки вы добавили и какие признаки удалили?

c) удалось ли добиться того, что алгоритм не переобучился и не недообучился?

In [ ]:
#your code here